<span style="font-width:bold; font-size: 3rem; color:#333;">- Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Getting the current date
today = datetime.date.today()
  
# Yesterday date
yesterday = today - datetime.timedelta(days = 1)

country="sweden"
city="stockholm"
street="stockholm-hornsgatan-108-gata"
AQI_API_KEY=""
str(yesterday)

'2024-02-17'

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [3]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store() 

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/5240
Connected. Call `.close()` to terminate connection gracefully.


---

## <span style='color:#ff5f27'> 🌫 Get Yesterday's Air Quality data (PM2.5) from the Feature Group</span>

In [4]:
# Read all data from the feature group
df_air_quality = air_quality_fg.read()

df = pd.DataFrame()
df_air_quality['day'] = pd.to_datetime(df_air_quality['date']).dt.date

row = df_air_quality.loc[df_air_quality['day'] == yesterday]

pm25_yesterday = row['pm25'].iloc[0]
pm25_yesterday

Finished: Reading data from Hopsworks, using ArrowFlight (0.57s) 


16.0

In [5]:
# Extracting the "date" and "city_name" columns from the 'df_air_quality' DataFrame
# Grouping the data by "city_name" and finding the maximum date for each city
last_dates_aq = df_air_quality[["date", "city", "street"]].groupby("street").max()

# Converting the date values to string format for consistency
last_dates_aq.date = last_dates_aq.date.astype(str)

In [6]:
import requests
import pandas as pd

# The API endpoint URL
url = f"https://api.waqi.info/feed/{country}/{street}/?token={AQI_API_KEY}"

# Make a GET request to fetch the data from the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the JSON content from the response
    data = response.json()
    
    # Check if the API response contains the data
    if data['status'] == 'ok':
        # Extract the air quality data
        aqi_data = data['data']
        aq_today_df = pd.DataFrame()  
        aq_today_df['pm25'] = [aqi_data['iaqi'].get('pm25', {}).get('v', None)]
        aq_today_df['pm25'] = aq_today_df['pm25'].astype('float32')
    else:
        print("Error: The API response does not contain data.")
else:
    print("Failed to retrieve data. Status Code:", response.status_code)
    
if pm25_yesterday is None: 
    print(f"Couldn't find yesterday's pm25 in your feature group. The latest value is from {df_air_quality['day'].max()}")
    # It is hard to get historical air quality data using aqicn.org, so we will just set it to today's value
    aq_today_df['pm25_yesterday'] = aq_today_df['pm25']
else:
    aq_today_df['pm25_yesterday'] = pm25_yesterday

aq_today_df['city'] = country
aq_today_df['street'] = street
aq_today_df['date'] = today
aq_today_df['date'] = pd.to_datetime(aq_today_df['date'])

aq_today_df

,pm25,pm25_yesterday,city,street,date
0,32.0,16.0,sweden,stockholm-hornsgatan-108-gata,2024-02-18


In [7]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pm25            1 non-null      float32       
 1   pm25_yesterday  1 non-null      float32       
 2   city            1 non-null      object        
 3   street          1 non-null      object        
 4   date            1 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float32(2), object(2)
memory usage: 160.0+ bytes


## <span style='color:#ff5f27'> 🌦 Filling gaps in Weather data</span>

In [8]:
# Read all data from the feature group
df_weather = weather_fg.read()
latest_date_weather_df = df_weather[["date", "city"]].groupby("city").max()
latest_date_weather = latest_date_weather_df['date'].iloc[0].date()
weather_df = get_historical_weather(city, str(latest_date_weather), str(today))
weather_df
weather_df = weather_df.dropna()
weather_df

Finished: Reading data from Hopsworks, using ArrowFlight (0.46s) 
Coordinates 59.29701232910156°N 18.163265228271484°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-02-16,3.43525,19.200003,17.477621,179.083359,stockholm


In [9]:
## Get weather forecst and write to feature group

hourly_df = get_hourly_weather_forecast(city)
hourly_df = hourly_df.set_index('date')
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df

Coordinates 59.19999694824219°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m
0,2024-02-18,2.50,0.0,6.489992,199.440125
1,2024-02-19,1.15,0.3,14.145756,165.256393
2,2024-02-20,1.30,0.0,7.289445,302.905243
3,2024-02-21,3.70,0.2,7.594207,185.440247
4,2024-02-22,2.50,0.0,14.007655,154.093552
5,2024-02-23,5.35,0.2,14.658676,245.323151
6,2024-02-24,4.90,0.0,16.622490,184.969650
7,2024-02-25,1.65,0.8,14.332341,168.407852
8,2024-02-26,5.45,0.0,7.517021,73.300674
9,2024-02-27,3.25,1.3,11.212135,47.602623


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-02-18 14:18:12,384 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/5240/fs/5188/fg/5170


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/5240/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f6b2c497e80>,
 {
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "aq_expectation_suite",
     "run_id": {
       "run_time": "2024-02-18T14:18:12.384037+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "8c7a85c4-ce68-11ee-9fb3-d67b32f142dc"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20240218T141812.383988Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "result": {
         "observed_value": 32.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-0

In [11]:
# Insert new data
weather_fg.insert(weather_df)

2024-02-18 14:18:17,189 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/5240/fs/5188/fg/5171


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/5240/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f6a629f8790>,
 {
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "weather_expectation_suite",
     "run_id": {
       "run_time": "2024-02-18T14:18:17.189606+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "8f57e200-ce68-11ee-9fb3-d67b32f142dc"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20240218T141817.189531Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "result": {
         "observed_value": 19.200002670288086,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "valid

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
